Today I'll try to predict wether the passenger survived or not during the Titanic Disaster

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv("/kaggle/input/titanic/train.csv")
test = pd.read_csv("/kaggle/input/titanic/test.csv")

Convert NaN into 0

In [ ]:
train = train.fillna(0)
test = test.fillna(0)

In [ ]:
train

I'm just want to inspect how many people survived from the disaster in the train dataset

In [ ]:
sns.countplot(train["Survived"])

It seems that the cabin code means something.

I think the first letter means something like the cabin position, so I'll group them by the first letter

In [ ]:
cabin_type = []
cab = train["Cabin"]
for x in cab:
    if str(x)[0:1] not in cabin_type:
        cabin_type.append(str(x)[0:1])
        
        
cabin_type



I also interested in Embarked colum

In [ ]:
embarked = []
col = train["Embarked"]
for x in col:
    if str(x)[0:1] not in embarked:
        embarked.append(str(x)[0:1])
        
        
embarked


Since I still not able to use the string feature, so I'm gonna convert them to integer that represents their class

In [ ]:

train["Cabin"] = [cabin_type.index(str(x)[0:1]) for x in train["Cabin"]]
test["Cabin"] = [cabin_type.index(str(x)[0:1]) for x in test["Cabin"]]

There's just a few people in the Cabin, 0 means NaN

In [ ]:
sns.countplot(train["Cabin"])

In [ ]:
train["Sex"] = [int(x == "male") for x in train["Sex"]]
test["Sex"] = [int(x == "male") for x in test["Sex"]]
#1 = male

I know that most of the Titanic survivor is Female, I'm gonna ensure that

In [ ]:
sns.countplot(train["Sex"])

In [ ]:
sns.countplot(train[train["Survived"] == 1]["Sex"])

The majority of Titanic Passengers are male but the survivors are mostly female.
(Ladies first, hats off)

Now I'm inspecting the Embarked column. I'm not sure what it is but I think that's the place where the Passenger get into the ship

In [ ]:
train["Embarked"] = [embarked.index(str(x)) for x in train["Embarked"]]
test["Embarked"] = [embarked.index(str(x)) for x in test["Embarked"]]

In [ ]:
sns.countplot(train["Embarked"])

Here's the dataframe after I changed the string features, I guess the Name and Ticket won't do anything so I'll drop it later

In [ ]:
train

Let's inspect the correlation

In [ ]:
plt.figure(figsize=(8,8))
sns.heatmap(train.corr(), annot=True)


In [ ]:
train.corr()["Survived"]

Let's drop the features with less correlation to the Survived column

In [ ]:
passenger_id = test["PassengerId"]
train = train.drop(["PassengerId", "Name", "Ticket", "Age", "SibSp", "Parch"], axis=1)
test = test.drop(["PassengerId", "Name", "Ticket", "Age", "SibSp", "Parch"], axis=1)

In [ ]:
train

In [ ]:
test

In [ ]:
X_train = train.drop(["Survived"], axis=1)
y_train = train["Survived"]


Since the test dataframe has no label, so let's put all of the columns to X_test

In [ ]:
X_test = test


# Logistic Regression

Let's try use the Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression
logr = LogisticRegression()
logr.fit(X_train, y_train)
logr.score(X_train, y_train)

In [ ]:
pred = logr.predict(X_test)
pred = [int(x >= 0.5) for x in pred]

This cell is to make the submission spreadsheet

In [ ]:
my_submission = pd.DataFrame({'PassengerId': passenger_id, 'Survived': pred})
# you could use any filename. We choose submission here
my_submission.to_csv('submission.csv', index=False)

# Deep Neural Network

I'm also trying to use Deep Neural Network

In [ ]:
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [ ]:
from keras.models import *
# from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Input
from keras.layers import Activation, Dropout, Flatten, Dense, BatchNormalization
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

model = Sequential()
model.add(Dense(32))
model.add(Activation("relu"))
model.add(Dense(256))
model.add(Activation("relu"))
model.add(Dense(256))
model.add(Activation("relu"))
model.add(Dense(1))
model.add(Activation("sigmoid"))

model.compile(optimizer = Adam(1e-4),
             loss = "binary_crossentropy",
             metrics = ["accuracy", f1_m])

In [ ]:
early_stop = EarlyStopping(patience=20, verbose=1, monitor='val_f1_m', mode='max')
reduce_lr =  ReduceLROnPlateau(monitor='val_f1_m', factor=0.1, patience=20, min_delta=1e-4, mode='max')

In [ ]:
model.fit(X_train, 
          y_train, 
          validation_split = 0.3,
          callbacks=[early_stop, reduce_lr],
          epochs = 500)

In [ ]:
pred_nn = model.predict(X_test)
pred_nn = [int(x >= 0.5) for x in pred_nn]
# model.score(X_train, y_train)

This also for the submission spreadsheet

In [ ]:
NN_sub = pd.DataFrame({'PassengerId': passenger_id, 'Survived': pred_nn})
# you could use any filename. We choose submission here
NN_sub.to_csv('submission_NN.csv', index=False)

Today I'm gonna try Random Forest and Gradient Boosting

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=100, max_depth=3, random_state=2)
rf.fit(X_train, y_train)
rf_pred = rf.predict(X_test)
rf.score(X_train, y_train)

In [ ]:
rf_sub = pd.DataFrame({'PassengerId': passenger_id, 'Survived': rf_pred})
# you could use any filename. We choose submission here
rf_sub.to_csv('submission_RF.csv', index=False)

this model scored 0.779

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
gb = GradientBoostingClassifier(learning_rate=0.06, n_estimators = 100,validation_fraction = 0.2, n_iter_no_change=100)
gb.fit(X_train, y_train)
gb.score(X_train, y_train)

In [ ]:
gb_pred =  gb.predict(X_test)

In [ ]:
gb_sub = pd.DataFrame({'PassengerId': passenger_id, 'Survived': gb_pred})
# you could use any filename. We choose submission here
gb_sub.to_csv('submission_GB.csv', index=False)

this model scored 0.775

# Conclusion

My Logistic Regression model is only scored around 0.75, and my Neural Network model is only scored around 0.73

Today I tried improving my Neuran Network model but it scored less, I also tried Random Forest and Gradient Boosting, I got 0.779 from Random Forest and 0.775 from Gradient Boosting

I was too naive to think that I can surpass 0.85 in the first or second try.

I guess I'll try another dataset tomorrow, I have to gather more experiences and skills, then improve these models again later...